<a href="https://colab.research.google.com/gist/hkrobotics/09794da09da38d3f03a31f6c00118b45/sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/hkrobotics/datasets-lp3/main/sales_data_sample.csv",sep=",", encoding='Latin-1')

In [61]:
df.drop(columns=["ADDRESSLINE1","ADDRESSLINE2","STATUS","POSTALCODE","CITY","TERRITORY","PHONE","STATE","CONTACTLASTNAME","CONTACTFIRSTNAME","ORDERNUMBER"],inplace=True)

In [62]:
productline = pd.get_dummies(df['PRODUCTLINE'])  
Dealsize = pd.get_dummies(df['DEALSIZE'])

In [63]:
df.drop(columns=["COUNTRY","ORDERDATE"],inplace=True)

In [64]:
df = pd.concat([df,productline,Dealsize],axis=1)

In [65]:
df.drop(columns=["PRODUCTLINE","DEALSIZE","CUSTOMERNAME"],inplace=True)

In [66]:
df['PRODUCTCODE'] = pd.Categorical(df['PRODUCTCODE']).codes

In [68]:
from sklearn.cluster import KMeans, k_means 
distortions = [] # Within Cluster Sum of Squares from the centroid
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df)
    distortions.append(kmeanModel.inertia_)  

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [70]:
X_train = df.values

In [71]:
model = KMeans(n_clusters=3,random_state=2) #Number of cluster = 3
model = model.fit(X_train) #Fitting the values to create a model.
predictions = model.predict(X_train) #Predicting the cluster values (0,1,or 2)

In [72]:
unique,counts = np.unique(predictions,return_counts=True)

In [73]:
counts = counts.reshape(1,3)

In [74]:
counts_df = pd.DataFrame(counts,columns=['Cluster1','Cluster2','Cluster3'])


In [75]:
counts_df.head()

,Cluster1,Cluster2,Cluster3
0,1083,1367,373


In [78]:
from sklearn.decomposition import PCA #Linear Dimensionality reduction.
pca = PCA(n_components=2) #Converting all the features into 2 columns to make it easy to visualize using Principal COmponent Analysis.

In [79]:
reduced_X = pd.DataFrame(pca.fit_transform(X_train),columns=['PCA1','PCA2']) #Creating a DataFrame.


In [ ]:
reduced_X.head()


In [ ]:
#Plotting the normal Scatter Plot
plt.figure(figsize=(14,10))
plt.scatter(reduced_X['PCA1'],reduced_X['PCA2'])

In [ ]:
model.cluster_centers_ #Finding the centriods. (3 Centriods in total. Each Array contains a centroids for particular feature )


In [ ]:
reduced_centers = pca.transform(model.cluster_centers_) #Transforming the centroids into 3 in x and y coordinates
plt.figure(figsize=(14,10))
plt.scatter(reduced_X['PCA1'],reduced_X['PCA2'])
plt.scatter(reduced_centers[:,0],reduced_centers[:,1],color='black',marker='x',s=300) #Plotting the centriods

In [85]:
reduced_X['Clusters'] = predictions #Adding the Clusters to the reduced dataframe.


In [ ]:
#Plotting the clusters 
plt.figure(figsize=(14,10))
#                     taking the cluster number and first column           taking the same cluster number and second column      Assigning the color
plt.scatter(reduced_X[reduced_X['Clusters'] == 0].loc[:,'PCA1'],reduced_X[reduced_X['Clusters'] == 0].loc[:,'PCA2'],color='slateblue')
plt.scatter(reduced_X[reduced_X['Clusters'] == 1].loc[:,'PCA1'],reduced_X[reduced_X['Clusters'] == 1].loc[:,'PCA2'],color='springgreen')
plt.scatter(reduced_X[reduced_X['Clusters'] == 2].loc[:,'PCA1'],reduced_X[reduced_X['Clusters'] == 2].loc[:,'PCA2'],color='indigo')


plt.scatter(reduced_centers[:,0],reduced_centers[:,1],color='black',marker='x',s=300)